In [3]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-09 22:27:28


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [4]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [5]:
# --- REMI V6.1: Softer Empathy Layer ---

import random
from textblob import TextBlob

# Emotion-aware response pools (less clinical, more human)
empathy_responses = {
    "bad": [
        "I'm sorry today feels bad.",
        "That sounds like a hard day.",
        "Some days are just heavy."
    ],
    "sad": [
        "That sounds really sad.",
        "I'm here with you.",
        "I'm glad you told me."
    ],
    "tired": [
        "Feeling tired can be exhausting.",
        "Maybe your body needs some rest.",
        "We can take things slowly."
    ],
    "lonely": [
        "Feeling lonely is really hard.",
        "You're not alone right now.",
        "I'm here with you."
    ],
    "angry": [
        "That sounds frustrating.",
        "It's okay to feel upset.",
        "I hear how strong that feeling is."
    ],
    "anxious": [
        "It's okay. You're safe right now.",
        "Let's slow down together.",
        "Nothing is urgent."
    ]
}

grounding_phrases = [
    "You're safe right now.",
    "Nothing needs to be done.",
    "We can just sit for a moment."
]

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False,
    "comfort_mode": False
}

print("✅ Remi V6.1 Empathy Layer Loaded")


✅ Remi V6.1 Empathy Layer Loaded


In [13]:
# --- REMI V10: The Honest Bot (Bug Fixes) ---

import random
import re
from datetime import datetime

chat_state = {
    "standing_by": False,
    "waiting_for_name": False,
    "user_profile": {"name": None}
}

# --- HELPER: Fixes the "nothing" vs "hi" bug ---
def has_word(word_list, text):
    """
    Checks for WHOLE words only. 
    'hi' will match 'hi there' but NOT 'nothing' or 'this'.
    """
    text = text.lower()
    for word in word_list:
        # \b means "word boundary" in Regex
        if re.search(r'\b' + re.escape(word) + r'\b', text):
            return True
    return False

def get_time_greeting():
    h = datetime.now().hour
    if h < 12: return "Good morning"
    elif 12 <= h < 18: return "Good afternoon"
    return "Good evening"

def process_response(user_input):
    text = user_input.lower().strip()
    
    # --- LAYER 1: WAITING FOR NAME ---
    if chat_state["waiting_for_name"]:
        # If user refuses
        if has_word(["no", "nope", "don't", "nothing", "stop"], text):
            chat_state["waiting_for_name"] = False
            return "Okay. I won't ask for your name. We can just chat."
        
        # Assume input is the name
        name = text.split()[0].capitalize() # Take first word
        chat_state["user_profile"]["name"] = name
        chat_state["waiting_for_name"] = False
        return f"Nice to meet you, {name}."

    # --- LAYER 2: HANDLING "UPSET" / "DON'T ASK" ---
    # This must come BEFORE greetings
    if has_word(["stop", "upset", "angry", "don't", "quiet"], text):
        chat_state["standing_by"] = True
        return "I hear you. I will stop asking questions now. I am here if you change your mind."

    # --- LAYER 3: REALITY CHECK (Weather/Location) ---
    if has_word(["rain", "raining", "sun", "sunny", "weather", "outside"], text):
        return "I live inside this computer, so I can't see the sky. But I can check the time! If you look out the window, what do you see?"

    if has_word(["time", "clock"], text):
        now = datetime.now().strftime("%I:%M %p")
        return f"My internal clock says it is {now}."

    # --- LAYER 4: GREETINGS (Now Fixed!) ---
    # Uses has_word so "nothing" doesn't trigger "hi"
    if has_word(["hello", "hi", "hey", "greetings"], text):
        greeting = get_time_greeting()
        name = chat_state["user_profile"]["name"]
        
        if name:
            return f"{greeting}, {name}."
        else:
            chat_state["waiting_for_name"] = True
            return f"{greeting}. I don't know your name yet. What is it?"

    # --- LAYER 5: ACTIVITIES ---
    if has_word(["walk", "walking"], text):
        return "A walk sounds lovely. Do you prefer walking in the city or the park?"
    
    if has_word(["hungry", "food", "eat"], text):
        return "You should get something to eat. I can wait here."

    # --- LAYER 6: FALLBACKS ---
    # If user says "nothing" or "no", just acknowledge it
    if has_word(["nothing", "no", "nope"], text):
        return "Okay. We can just sit quietly."

    return "I am listening."

# -----------------------------
# INTERACTIVE LOOP
# -----------------------------
print("--- Remi V10 (Bug Fixed) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break
        
    response = process_response(user_input)
    print(f"Remi: {response}")

--- Remi V10 (Bug Fixed) ---


You:  knock knock


Remi: I am listening.


You:  hello


Remi: Good evening. I don't know your name yet. What is it?


You:  I am peter


Remi: Nice to meet you, I.


You:  what is your name?


Remi: I am listening.


KeyboardInterrupt: Interrupted by user

The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.

We now want to introduce long term memory to our bot

For the next phases, we will do the following: 
Phase 1 (Done): Rule-Based Logic and it's about understanding how input/output works.

Phase 2 (NOW): Data Creation. We need to create a "textbook" to teach the bot.

Phase 3 (Next): Vectorization. Converting words into numbers (Math).

Phase 4 (The Goal): Training a Neural Network. Using TensorFlow/Keras to build a model that learns from your data.

Step 1: Creating the "Textbook" (Training Data)
In Machine Learning, we don't write if statements. We create Intents.

Intent: What the user means (e.g., greeting, sadness, reminiscence_childhood).

Patterns: Examples of how humans say it.

Responses: What the bot should say back.

We need to create a file called intents.json. This will be the source code for your AI's brain.

In [ ]:
import json

# This is the "Textbook" we will feed to the Neural Network
data = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up", "Hey there"],
      "responses": ["Hello!", "Good to see you again", "Hi there, how can I help?"],
      "context": [""]
    },
    {
      "tag": "goodbye",
      "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "quit"],
      "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],
      "context": [""]
    },
    {
      "tag": "identity",
      "patterns": ["who are you", "what are you", "what is your name", "are you human"],
      "responses": ["I am Remi, your memory companion.", "I am a bot designed to help you share your stories."],
      "context": [""]
    },
    {
      "tag": "mood_sad",
      "patterns": ["I feel sad", "I am lonely", "I feel bad", "I am upset", "Everything is wrong", "I am depressed"],
      "responses": ["I am sorry to hear that. I am here to listen.", "It's okay to feel down sometimes. Do you want to talk about it?"],
      "context": [""]
    },
    {
      "tag": "reminisce_childhood",
      "patterns": ["I was young", "When I was a kid", "My childhood", "I grew up in", "school days"],
      "responses": ["Childhood memories are precious. What was your favorite game to play back then?", "Tell me about the house you grew up in."],
      "context": [""]
    },
    {
      "tag": "confusion",
      "patterns": ["Where am I?", "I don't know who I am", "I am lost", "What is happening"],
      "responses": ["You are safe. You are chatting with Remi.", "Take a deep breath. You are safe right here."],
      "context": [""]
    }
  ]
}

# Save this data to a file on your D: drive
with open('intents.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("✅ Training Data Created: 'intents.json'")
print("We have stopped writing rules. We are now ready for Machine Learning.")